# Baseline vs Models - MIMIC Dataset - Boostrap Test Set

In [1]:
import pandas as pd
import numpy as np
import yaml
import os
from importlib import resources as impresources
from recurrent_health_events_prediction import configs
import plotly.express as px
from recurrent_health_events_prediction.model_analysis.utils import MetricEnum

In [2]:
with open((impresources.files(configs) / 'model_config.yaml')) as f:
    model_config = yaml.safe_load(f)
with open((impresources.files(configs) / 'data_config.yaml')) as f:
    data_config = yaml.safe_load(f)

In [3]:
SAVE_RESULTS_DIR = data_config['final_results']["mimic_path"]

In [4]:
DATASET = "mimic"
BASELINE_RESULTS_DIR = "/workspaces/master-thesis-recurrent-health-events-prediction/_models/mimic/classifiers_baselines"
BASE_MODEL_DIR = model_config[DATASET]["hidden_markov"]["save_model_path"]
CLASSIFIER = "lgbm"
METRIC = MetricEnum.AUC.value
N_BOOTSTRAPS = 5000
RANDOM_STATE = 42
THRESHOLD = 0.5 # probability threshold for classification

In [5]:
filename = f"hmm_vs_baseline_{DATASET}_{CLASSIFIER}_{METRIC}.csv"
print(f"Saving results to {os.path.join(SAVE_RESULTS_DIR, filename)}")

Saving results to /workspaces/master-thesis-recurrent-health-events-prediction/results/mimic/hmm_vs_baseline_mimic_lgbm_auc.csv


## hmm_mimic_time_log_normal_v1

In [6]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_mimic_time_log_normal_v1"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'event_id_col': 'HADM_ID',
 'event_name': 'readmission',
 'event_time_cat_col': 'READMISSION_TIME_CAT',
 'event_time_feature_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'feature_define_state_labels': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'features': {'DISCHARGE_LOCATION_POST_ACUTE_CARE': 'bernoulli',
  'LOG_DAYS_IN_ICU': 'gaussian',
  'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION': 'gaussian',
  'LOG_NUM_DRUGS': 'gaussian',
  'CHARLSON_INDEX': 'poisson'},
 'hidden_state_labels': ['high', 'medium', 'low'],
 'id_col': 'SUBJECT_ID',
 'model_name': 'hmm_mimic_time_log_normal_v1',
 'n_states': 3,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/mimic/hmm',
 'time_col': 'ADMITTIME',
 'use_only_sequences_gte_2_steps': False}

In [7]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results1_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_mimic_time_log_normal_v1 with baseline using auc metric...
Number of predictions in baseline: 274
Number of predictions in hmm_mimic_time_log_normal_v1: 274
Number of common predictions: 274
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



  2%|▏         | 102/5000 [00:00<00:09, 507.18it/s]

100%|██████████| 5000/5000 [00:10<00:00, 498.63it/s]


In [8]:
results1_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_mimic_time_log_normal_v1,3,lgbm,auc,-0.000347,-0.000745,-0.080344,0.080004,0.9884,5000


## hmm_mimic_reduced_feats_set

In [9]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config

model_name = "hmm_mimic_reduced_feats_set"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'event_id_col': 'HADM_ID',
 'event_name': 'readmission',
 'event_time_cat_col': 'READMISSION_TIME_CAT',
 'event_time_feature_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'feature_define_state_labels': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'features': {'CHARLSON_INDEX': 'poisson',
  'DISCHARGE_LOCATION_POST_ACUTE_CARE': 'bernoulli',
  'LOG_DAYS_IN_ICU': 'gaussian',
  'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION': 'gaussian',
  'LOG_NUM_DRUGS': 'gaussian'},
 'hidden_state_labels': ['high', 'medium', 'low'],
 'id_col': 'SUBJECT_ID',
 'model_name': 'hmm_mimic_reduced_feats_set',
 'n_states': 3,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/mimic/hmm',
 'time_col': 'ADMITTIME',
 'use_only_sequences_gte_2_steps': False}

In [10]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results_reduced_set_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_mimic_reduced_feats_set with baseline using auc metric...
Number of predictions in baseline: 274
Number of predictions in hmm_mimic_reduced_feats_set: 274
Number of common predictions: 274
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:11<00:00, 441.90it/s]


In [11]:
results_reduced_set_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_mimic_reduced_feats_set,3,lgbm,auc,-0.035038,-0.036026,-0.129746,0.061483,0.4512,5000


## hmm_mimic_time_log_normal_v2

In [12]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_mimic_time_log_normal_v2"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'event_id_col': 'HADM_ID',
 'event_name': 'readmission',
 'event_time_cat_col': 'READMISSION_TIME_CAT',
 'event_time_feature_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'feature_define_state_labels': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'features': {'HAS_COPD': 'bernoulli',
  'LOG_DAYS_IN_ICU': 'gaussian',
  'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION': 'gaussian',
  'LOG_NUM_DRUGS': 'gaussian',
  'NUM_COMORBIDITIES': 'poisson'},
 'hidden_state_labels': ['high_2', 'high_1', 'medium', 'low_2', 'low_1'],
 'id_col': 'SUBJECT_ID',
 'model_name': 'hmm_mimic_time_log_normal_v2',
 'n_states': 5,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/mimic/hmm',
 'time_col': 'ADMITTIME',
 'use_only_sequences_gte_2_steps': False}

In [13]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results6_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_mimic_time_log_normal_v2 with baseline using auc metric...
Number of predictions in baseline: 274
Number of predictions in hmm_mimic_time_log_normal_v2: 274
Number of common predictions: 274
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



  0%|          | 21/5000 [00:00<00:23, 207.68it/s]

100%|██████████| 5000/5000 [00:11<00:00, 430.80it/s]


In [14]:
results6_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_mimic_time_log_normal_v2,5,lgbm,auc,-0.014709,-0.014602,-0.084715,0.055297,0.678,5000


## hmm_mimic_time_log_gamma

In [15]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_mimic_time_log_gamma"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'event_id_col': 'HADM_ID',
 'event_name': 'readmission',
 'event_time_cat_col': 'READMISSION_TIME_CAT',
 'event_time_feature_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'feature_define_state_labels': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'features': {'CHARLSON_INDEX': 'poisson',
  'DISCHARGE_LOCATION_POST_ACUTE_CARE': 'bernoulli',
  'LOG_DAYS_IN_ICU': 'gaussian',
  'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION': 'gamma'},
 'hidden_state_labels': ['high', 'medium', 'low'],
 'id_col': 'SUBJECT_ID',
 'model_name': 'hmm_mimic_time_log_gamma',
 'n_states': 3,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/mimic/hmm',
 'time_col': 'ADMITTIME',
 'use_only_sequences_gte_2_steps': False}

In [16]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results2_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_mimic_time_log_gamma with baseline using auc metric...
Number of predictions in baseline: 274
Number of predictions in hmm_mimic_time_log_gamma: 274
Number of common predictions: 274
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



100%|██████████| 5000/5000 [00:10<00:00, 459.25it/s]


In [17]:
results2_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_mimic_time_log_gamma,3,lgbm,auc,-0.042254,-0.042198,-0.102546,0.017349,0.1628,5000


## hmm_mimic_time_binary_30_days

In [18]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_mimic_time_binary_30_days"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'READMISSION_30_DAYS',
 'event_id_col': 'HADM_ID',
 'event_name': 'readmission',
 'event_time_cat_col': 'READMISSION_TIME_CAT',
 'event_time_feature_col': 'READMISSION_30_DAYS',
 'feature_define_state_labels': 'READMISSION_30_DAYS',
 'features': {'DISCHARGE_LOCATION_POST_ACUTE_CARE': 'bernoulli',
  'LOG_DAYS_IN_ICU': 'gaussian',
  'LOG_NUM_DRUGS': 'gaussian',
  'NUM_COMORBIDITIES': 'poisson',
  'READMISSION_30_DAYS': 'bernoulli'},
 'hidden_state_labels': ['low', 'medium_1', 'medium_2', 'high'],
 'id_col': 'SUBJECT_ID',
 'model_name': 'hmm_mimic_time_binary_30_days',
 'n_states': 4,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/mimic/hmm',
 'time_col': 'ADMITTIME',
 'use_only_sequences_gte_2_steps': False}

In [19]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results3_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_mimic_time_binary_30_days with baseline using auc metric...
Number of predictions in baseline: 274
Number of predictions in hmm_mimic_time_binary_30_days: 274
Number of common predictions: 274
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:09<00:00, 528.49it/s]


In [20]:
results3_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_mimic_time_binary_30_days,4,lgbm,auc,-0.021578,-0.022402,-0.097551,0.053079,0.5768,5000


## hmm_mimic_time_log_normal_min_emissions

In [21]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_mimic_time_log_normal_min_emissions"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'event_id_col': 'HADM_ID',
 'event_name': 'readmission',
 'event_time_cat_col': 'READMISSION_TIME_CAT',
 'event_time_feature_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'feature_define_state_labels': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'features': {'DISCHARGE_LOCATION_POST_ACUTE_CARE': 'bernoulli',
  'LOG_DAYS_IN_ICU': 'gaussian',
  'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION': 'gaussian'},
 'hidden_state_labels': ['high', 'medium_2', 'medium_1', 'low'],
 'id_col': 'SUBJECT_ID',
 'model_name': 'hmm_mimic_time_log_normal_min_emissions',
 'n_states': 4,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/mimic/hmm',
 'time_col': 'ADMITTIME',
 'use_only_sequences_gte_2_steps': False}

In [22]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results4_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_mimic_time_log_normal_min_emissions with baseline using auc metric...
Number of predictions in baseline: 274
Number of predictions in hmm_mimic_time_log_normal_min_emissions: 274
Number of common predictions: 274
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



  1%|▏         | 66/5000 [00:00<00:15, 315.81it/s]

100%|██████████| 5000/5000 [00:10<00:00, 460.16it/s]


In [23]:
results4_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_mimic_time_log_normal_min_emissions,4,lgbm,auc,-0.000763,-0.001161,-0.080485,0.079033,0.9748,5000


## hmm_mimic_time_log_student_t

In [24]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_mimic_time_log_student_t"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'dof': 5,
 'event_id_col': 'HADM_ID',
 'event_name': 'readmission',
 'event_time_cat_col': 'READMISSION_TIME_CAT',
 'event_time_feature_col': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'feature_define_state_labels': 'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION',
 'features': {'DISCHARGE_LOCATION_POST_ACUTE_CARE': 'bernoulli',
  'LOG_DAYS_IN_ICU': 'student_t',
  'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION': 'student_t',
  'LOG_NUM_DRUGS': 'gaussian',
  'NUM_COMORBIDITIES': 'poisson'},
 'hidden_state_labels': ['high', 'medium', 'low'],
 'id_col': 'SUBJECT_ID',
 'model_name': 'hmm_mimic_time_log_student_t',
 'n_states': 3,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/mimic/hmm',
 'time_col': 'ADMITTIME',
 'use_only_sequences_gte_2_steps': False}

In [25]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results5_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_mimic_time_log_student_t with baseline using auc metric...
Number of predictions in baseline: 274
Number of predictions in hmm_mimic_time_log_student_t: 274
Number of common predictions: 274
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:11<00:00, 426.90it/s]


In [26]:
results5_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_mimic_time_log_student_t,3,lgbm,auc,-0.03483,-0.035229,-0.104631,0.033237,0.324,5000


## All Results

In [27]:
all_results_df_list = [results_reduced_set_df, results1_df, results2_df, results3_df, results4_df, results5_df, results6_df]
all_results_df = pd.concat(all_results_df_list, ignore_index=True).sort_values(by="delta_mean")


In [28]:
all_results_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
2,hmm_mimic_time_log_gamma,3,lgbm,auc,-0.042254,-0.042198,-0.102546,0.017349,0.1628,5000
0,hmm_mimic_reduced_feats_set,3,lgbm,auc,-0.035038,-0.036026,-0.129746,0.061483,0.4512,5000
5,hmm_mimic_time_log_student_t,3,lgbm,auc,-0.034830,-0.035229,-0.104631,0.033237,0.3240,5000
3,hmm_mimic_time_binary_30_days,4,lgbm,auc,-0.021578,-0.022402,-0.097551,0.053079,0.5768,5000
6,hmm_mimic_time_log_normal_v2,5,lgbm,auc,-0.014709,-0.014602,-0.084715,0.055297,0.6780,5000
4,hmm_mimic_time_log_normal_min_emissions,4,lgbm,auc,-0.000763,-0.001161,-0.080485,0.079033,0.9748,5000
1,hmm_mimic_time_log_normal_v1,3,lgbm,auc,-0.000347,-0.000745,-0.080344,0.080004,0.9884,5000


In [29]:
if not os.path.exists(SAVE_RESULTS_DIR):
    os.makedirs(SAVE_RESULTS_DIR, exist_ok=True)
all_results_df.to_csv(os.path.join(SAVE_RESULTS_DIR, filename), index=False)